In [57]:
import shutil

source_file_path = '.\..\..\.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

'.\\.env'

In [84]:
from bingx.api import BingxAPI
from dotenv import dotenv_values

import sys
project_path = '../../'  # Adjust this to your actual project path
sys.path.append(project_path)

from Shared.loadOHLC_FromJson import loadOHLC_FromJson
from Shared.findRiskToReward import findRiskToReward
# from Shared.saveOHLC_toJson import saveOHLC_toJson
# from PostAnalyzer.Utility.utils import print_colored
from PostAnalyzer.Utility.utils import load_json, convertTimestampToDateTime, convertToJsonFile, getNowTimestamp
from operator import itemgetter

config = dotenv_values(".env")

API_KEY = config["API_KEY"]
SECRET_KEY = config["SECRET_KEY"]

bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")
historic_json_path = "./../historic-json"

In [59]:
async def saveOHLC_toJson(symbolName, path, data):
    unique_data = {}
    for item in data:
        if item['time'] not in unique_data:
            unique_data[item['time']] = item

    # Convert the dictionary back to a list
    unique_list = list(unique_data.values())
    
    # Sort the unique data by the "time" key
    unique_list.sort(key=itemgetter('time'))
    
    convertToJsonFile(unique_list, symbolName, path) 

In [60]:
async def updateOHLC_FromAPI(start_timestamp, symbolName):

    LData = load_json(f"./2x.json")
    # LData = loadOHLC_FromJson(symbolName=symbolName, path=historic_json_path)

    keepOn = True
    time_interval = "1m" 
    # 60 * 24 = 1440 minutes in a day
    limit = 1440
    # 86400000 == milliseconds in a day
    milliSecInDay = 86400000    
    # 60_000 == milliseconds in a minute
    milliSecInMinute = 60_000

    allAPIdata = []

    # merge json data to api data
    if LData:
        first_item_time = LData[0]['time']
        last_item_time = LData[len(LData)-1]['time']

        # get older data
        if start_timestamp <( first_item_time - milliSecInMinute):
            end_time = first_item_time
            next_day = start_timestamp + milliSecInDay

            while keepOn:
                # print(3)
                res = bingx.get_kline_data(symbolName, time_interval, start_timestamp, next_day, limit)
                allAPIdata += list(reversed(res))

                start_timestamp = next_day
                next_day = start_timestamp + milliSecInDay

                if allAPIdata[len(allAPIdata)-1]['time'] > end_time:
                    keepOn = False
                    break
                if len(res) < limit:
                    keepOn = False
                    break


        # If the difference between the last item and the current time was more than one day, request to get new data
        if (getNowTimestamp() - last_item_time) > milliSecInDay:
            # api to get newer data
            keepOn = True
            start_timestamp = last_item_time - milliSecInMinute
            next_day = start_timestamp + milliSecInDay

            while keepOn:
                
                res = bingx.get_kline_data(symbolName, time_interval, last_item_time, next_day, limit)
                allAPIdata += list(reversed(res)) 

                start_timestamp = next_day
                next_day = start_timestamp + milliSecInDay
                
                if len(res) < limit:
                    keepOn = False
                    break

    # json data is empty, so fill with api data until now
    else:
        next_day = start_timestamp + milliSecInDay 
        while keepOn:
            # print(1)
            res = bingx.get_kline_data(symbolName, time_interval, start_timestamp, next_day, limit)
            allAPIdata += list(reversed(res)) 

            start_timestamp = next_day
            next_day = start_timestamp + milliSecInDay
            
            if len(res) < limit:
                keepOn = False
                break

    if allAPIdata:
       await saveOHLC_toJson("2x", "./", LData + allAPIdata)


In [61]:
# # SHORT
# from bingx.api import BingxAPI
# from dotenv import dotenv_values
# from PostAnalyzer.Utility.utils import returnSearchValue,convertToJsonFile,subtractTime,load_json,print_colored

# config = dotenv_values(".env")


# API_KEY = config["API_KEY"]
# SECRET_KEY = config["SECRET_KEY"]

# bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")


# symbol = "BTC-USDT" 
# time_interval = "1m"
# start_timestamp = 1717744080000 
# end_timestamp = 1720607400000 

# # 86400000 == milliseconds in a day
# next_day = start_timestamp + 86400000

# xxx = []

# keepOn = True
# limit = 1440
# milliSecInDay = 86400000

# while keepOn:
#     res = bingx.get_kline_data(symbol, time_interval ,start_timestamp ,next_day, limit)
#     xxx += list(reversed(res))

#     start_timestamp = next_day
#     next_day = start_timestamp + milliSecInDay

#     if res[len(res)-1]['time'] > end_timestamp:
#         keepOn = False

#     if len(res) < limit:
#         keepOn = False



# # convertToJsonFile(xxx,"2x",".")




In [63]:
entry_price = [63810, 64804]
stop_loss = 65395 
take_profit = [63256, 62373 , 61736, 60955, 60051]
symbol = "BTC-USDT" 

#   May 21, 2024 9:50:00 AM = 1716272400000
# start_timestamp = 1716272400000

# #  June 4, 2024 8:00:00 AM = 1717475400000
# start_timestamp = 1717475400000


start_timestamp = 1721136180000

await updateOHLC_FromAPI(start_timestamp, symbol)

LData = load_json(f"./2x.json")

tp_turn = 0
tp = take_profit[tp_turn]

entry_turn = 0
entry = entry_price[entry_turn]
entry_reached = []
tps = []
stop_loss_reached = None

for row in LData:
    if row['time'] < start_timestamp:
        continue

    if not entry_reached and float(row['high']) >= float(entry):
        entry_reached.append(row)

        if entry_turn < len(entry_price)-1:
            entry_turn += 1
            entry = entry_price[entry_turn]
        else:
            entry = float("inf")

        continue

    if entry_reached and float(row['high']) >= float(entry):
        entry_reached.append(row)

        if entry_turn < len(entry_price)-1:
            entry_turn += 1
            entry = entry_price[entry_turn]
        else:
            entry = float("inf")

        continue


    if bool(entry_reached):
        if not bool(stop_loss_reached) and float(row['high']) >= float(stop_loss):
            
            stop_loss_reached = row
            break

        if float(row['low']) <= float(tp):
            
            tps.append(row)

            if (tp_turn + 1) == len(take_profit):
                break
            tp_turn += 1
            tp = take_profit[tp_turn]
        

print(tps, entry_reached, stop_loss_reached)

for x in tps:
    print(convertTimestampToDateTime(x['time']))
    
print("*******************************entry***********************************")
for x in entry_reached:
    print(convertTimestampToDateTime(x['time']))   

print("*********************************stop_loss**********************************")
print(convertTimestampToDateTime(stop_loss_reached['time']))
 

[{'open': '63403.8', 'close': '63216.6', 'high': '63433.8', 'low': '63159.2', 'volume': '208.92', 'time': 1721138520000}] [{'open': '63792.6', 'close': '63775.8', 'high': '63810.0', 'low': '63759.2', 'volume': '17.09', 'time': 1721136420000}, {'open': '64648.3', 'close': '64808.8', 'high': '64856.5', 'low': '64636.5', 'volume': '77.12', 'time': 1721145120000}] {'open': '65347.0', 'close': '65337.8', 'high': '65437.6', 'low': '65332.2', 'volume': '80.82', 'time': 1721176320000}
july 16, 2024, 05:32 p.m
*******************************entry***********************************
july 16, 2024, 04:57 p.m
july 16, 2024, 07:22 p.m
*********************************stop_loss**********************************
july 17, 2024, 04:02 a.m


In [86]:
entry_price = [6.609, 6.797]
stop_loss = 0.6025 
# stop_loss = 6.900
take_profit = [6.535, 6.453 , 6.324, 6.204, 6.025]
symbol = "ATOM-USDT" 

print(findRiskToReward(entry_price, take_profit, stop_loss))


error
0.050079080995588096
